In [ ]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
from datetime import timedelta
from matplotlib.ticker import MaxNLocator

In [ ]:
# plotting parameters
figure_size = (6.25984252, 4.313975656) # = 15.9, 10.957 cm  (10.95749816624)

def rgb(rgb_255_tuple):
    return tuple(v/255 for v in rgb_255_tuple)

colour_op = rgb((13, 103, 53))
colour_s = rgb((254, 205, 8))
colour_p = rgb((237, 29, 36))

sns.set_palette('muted')

# Raw data processing

# Inflow profile calculation

In [ ]:
df = pd.read_csv('K7 data 1min 09-15_2017-09-26.pdi.gz')

# also make forward filled
time_interval = pd.Timedelta(minutes=1)
df = df.pivot_table(values='Value', columns='TagName', index=(pd.to_datetime(df['Date'].str.cat(df['Time'], sep=' ')) - time_interval))
df.index.name = 'DateTime'

df['Time_30'] = df.index.floor('30min').time

df['EskomDayOfWeek'] = df.index.dayofweek + 1
# replace holidays with Eskom days
h_path = '..\holidays.csv'
holidays = np.genfromtxt(h_path,  dtype=np.dtype('M'), delimiter=',', usecols=0)
holidays = list(holidays)
holidays_numbers = np.genfromtxt(h_path, dtype=int, delimiter=',', usecols=1)
holidays_numbers = list(holidays_numbers)
for h_n, h_d in zip(holidays_numbers, holidays):
    df.loc[df.index.date==h_d,'EskomDayOfWeek'] = h_n
# drop weekends
df = df[df['EskomDayOfWeek'] <= 5]
df.drop('EskomDayOfWeek', axis=1)

df.head(5)

In [ ]:
df = df.rename(columns = {'KDCE_S07_00INS00_00ILT#501.FA_PV': 'Surface Level'})

df = df.rename(columns = {'KDCE_S07_41PMP00_00ILT#501.FA_PV': '41L Level'})

df = df.rename(columns = {'KDCE_S07_31PMP00_00ILT#501.FA_PV': '31L Level'})

df = df.rename(columns = {'KDCE_S07_20PMP00_00ILT#501.FA_PV': '20L Level'})
#df.drop([col for col in df.columns if 'KDCE_S07_20PMP' in col], axis=1, inplace=True)

df['IPC Level'] = df[['KDCE_S07_IMPMP00_00ILT#501.FA_PV', 'KDCE_S07_IMPMP00_00ILT#502.FA_PV']].sum(axis=1)
df.drop(['KDCE_S07_IMPMP00_00ILT#501.FA_PV', 'KDCE_S07_IMPMP00_00ILT#502.FA_PV'], axis=1, inplace=True)

In [ ]:
status_col_dict = {}
for c in df.columns:
    if '_Machine.FA_RF' in c:
        item = c.split('_')[2].split('PMP')
        level = item[0] + 'L'
        level = 'IPC' if level == 'IML' else level
        pump_num = 'P' + item[1][1]
        status_col_dict[c] = level + ' ' + pump_num
        
df = df.rename(columns = status_col_dict)

In [ ]:
df.head(5)

## Process selected data

In [ ]:
def calculate_inflows(level_name, pump_flowrates, capacity, feeding_level_name=None):
    # feeding_level_name is the level feeding this level
    
    number_of_pumps = len(pump_flowrates)

    calc_pump_flows = [np.nan]
    calc_inflows = [np.nan]

    for i in range(1, len(df.index)):
        pump_status = {}
        for j,_ in enumerate(pump_flowrates):
            pump_str = 'P' + str(j+1)
            pump_status[pump_str] = df[level_name + ' ' + pump_str].values[i]
        
        l_new = df[level_name + ' Level'].values[i]
        l_old = df[level_name + ' Level'].values[i-1]
        pump_flow_from_prev_lev = 0 if feeding_level_name is None else df[feeding_level_name + ' PumpFlow'].values[i]
        
        if np.isnan([v for k,v in pump_status.items()] + [l_new] + [l_old] + [pump_flow_from_prev_lev]).any():
            ans_pumpflow = np.nan
            ans_inflow = np.nan

        delta_level = l_new - l_old
        if delta_level != 0:
            ans_outflow_pumps = 0
            for ps, pf in zip(pump_status.items(), pump_flowrates):
                ans_outflow_pumps += ps[1] * pf
                
            ans_inflow = ((l_new - l_old) / 100 * capacity) / 60 + ans_outflow_pumps - pump_flow_from_prev_lev
        else:
            ans_outflow_pumps = np.nan
            ans_inflow = np.nan

        calc_pump_flows.append(ans_outflow_pumps)
        calc_inflows.append(ans_inflow)

    df[level_name + ' PumpFlow'] = calc_pump_flows
    df[level_name + ' Inflow'] = calc_inflows

In [ ]:
calculate_inflows('41L', [231.4]*5, 3000000)
calculate_inflows('31L', [167.1]*4, 3000000, '41L')
calculate_inflows('20L', [263.7]*4, 3000000, '31L')
calculate_inflows('IPC', [358.5]*5, 3000000, '20L')

In [ ]:
# create dummy column for simulating surface
df['Surface P1'] = [0] * len(df.index)
calculate_inflows('Surface', [0]*1, 5000000, 'IPC')

In [ ]:
df.head(5)

# Export data for use in simulation

## Inflow profile

In [ ]:
df2 = df.copy()

date_start = '2016-09-15'
date_end = '2016-09-16'

df2 = df2[(df2.index>=date_start) & (df2.index<date_end)]

inflow_profiles = []
overall_day_completeness_count = 0
overall_day_completeness_max = 0
overall_completeness_count = 0
overall_completeness_max = 0
for l in ['41L', '31L', '20L', 'IPC', 'Surface']:
    grouped = df2.set_index('Time_30')[l + ' Inflow'].groupby('Time_30')
    
    grouped_mean = grouped.mean()
    inflow_profiles.append(grouped_mean)
    
    print('{} completeness: {:6.2f}% → {:6.2f}%'.format(l, grouped.count().sum()/48/30*100, grouped_mean.count()/48*100))#grouped.count()/30
    overall_day_completeness_count += grouped.count().sum()
    overall_day_completeness_max += 48*30
    overall_completeness_count += grouped_mean.count()
    overall_completeness_max += 48
print('                   ------   -------')
print('All completenes:  {:6.2f}% → {:6.2f}%'.format(overall_day_completeness_count/overall_day_completeness_max*100, overall_completeness_count/overall_completeness_max*100))
    
pd.DataFrame(inflow_profiles).T.to_csv('..\..\simulations\K7\input\K7_dam_inflow_profiles.csv.gz', compression='gzip')

## Data for verification of the simulation

In [ ]:
df_real = pd.read_csv('K7 data 1s 09-15_2017-09-26_pivot.csv.gz')
df_real = df_real.set_index('DateTime')
df_real.index = pd.to_datetime(df_real.index)
df_real.head(5)

In [ ]:
df_real['41L Status'] = df_real[[col for col in df_real.columns if ('41PMP' in col and 'Machine.FA_RF' in col)]].sum(skipna=False, axis=1)
df_real = df_real.rename(columns = {'KDCE_S07_41PMP00_00ILT#501.FA_PV': '41L Level'})
df_real.drop([col for col in df_real.columns if 'KDCE_S07_41PMP' in col], axis=1, inplace=True)

df_real['31L Status'] = df_real[[col for col in df_real.columns if ('31PMP' in col and 'Machine.FA_RF' in col)]].sum(skipna=False, axis=1)
df_real['31L Level'] = df_real[['KDCE_S07_31PMP00_00ILT#501.FA_PV', 'KDCE_S07_31PMP00_01ILT#501.FA_PV']].mean(axis=1)
df_real.drop([col for col in df_real.columns if 'KDCE_S07_31PMP' in col], axis=1, inplace=True)

df_real['20L Status'] = df_real[[col for col in df_real.columns if ('20PMP' in col and 'Machine.FA_RF' in col)]].sum(skipna=False, axis=1)
df_real = df_real.rename(columns = {'KDCE_S07_20PMP00_00ILT#501.FA_PV': '20L Level'})
df_real.drop([col for col in df_real.columns if 'KDCE_S07_20PMP' in col], axis=1, inplace=True)
df_real.drop([col for col in df_real.columns if 'KDCE_S07_22PMP' in col], axis=1, inplace=True)

df_real['IPC Status'] = df_real[[col for col in df_real.columns if ('IMPMP' in col and 'Machine.FA_RF' in col)]].sum(skipna=False, axis=1)
df_real['IPC Level'] = df_real[['KDCE_S07_IMPMP00_00ILT#501.FA_PV', 'KDCE_S07_IMPMP00_00ILT#502.FA_PV']].sum(axis=1)
df_real.drop([col for col in df_real.columns if 'KDCE_S07_IMPMP' in col], axis=1, inplace=True)

df_real = df_real.rename(columns = {'KDCE_S07_00INS00_00ILT#501.FA_PV': 'Surface Level'})

df_real.head(5)

In [ ]:
date_range_start = '2016-09-15'
date_range_end = '2016-09-16'
df_real2 = df_real.ix[date_range_start:date_range_end]

level_list = ['41L', '31L', '20L', 'IPC', 'Surface']
level_tag_list = [l + ' Level' for l in level_list]
status_tag_list = [l + ' Status' for l in level_list if l is not 'Surface']
tag_list = level_tag_list + status_tag_list

df_real2[level_tag_list].plot()
df_real2[status_tag_list].plot()
plt.show()
plt.close('all')

#### Save simulation testing data

In [ ]:
df_real2[tag_list].to_csv('..\..\simulations\K7\input\K7_data_for_verification.csv.gz', compression='gzip')

# Now perform the simulation in verification mode

 **If the simulation outputs are reasonably "the same" as the real data, the simulation is "correct"**
 
 Run the simulation in verification_mode using initial values for dam level and following pump status instead of scheduler

In [ ]:
df_sim = pd.read_csv('..\..\simulations\K7\output\K7_simulation_data_export_verification.csv.gz')
df_sim['time_clean'] = pd.to_datetime(df_sim['seconds'], unit='s') + timedelta(days=17059)
df_sim.head(5)

In [ ]:
def rmse(real, sim):
    return np.sqrt(((real - sim) ** 2).mean())

def r2(x, y):
    return stats.pearsonr(x,y)[0] ** 2

## Comparison of simulation and actual data

### 41L pump status

In [ ]:
level = '41L'

In [ ]:
x = pd.to_datetime(df_real2.index.values).time
y1 = df_real2[level + ' Status'].values
y2 = df_sim[level + ' Status'].values

print('R squared = ', r2(y1, y2))
print('RMSE = ', rmse(y1, y2))

In [ ]:
fig, ax1 = plt.subplots(figsize=(figure_size[0], figure_size[1]/1.5), dpi=1200)

x = df_sim['time_clean']

ax1.plot(x, df_real2[level + ' Status'], marker=None, label="Actual data", zorder=10)
ax1.plot(x, df_sim[level + ' Status'], marker=None, label="Simulation data", zorder=11, color=sns.color_palette('muted')[2])

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Number of pumps running')
ax1.set_ylim([0, 3])
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))

ax1.axvspan('2016-09-15 00:00:00','2016-09-15 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('2016-09-15 06:00:00','2016-09-15 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 07:00:00','2016-09-15 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 10:00:00','2016-09-15 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 18:00:00','2016-09-15 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 20:00:00','2016-09-15 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 22:00:00','2016-09-15 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x),max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)

fig.tight_layout()
plt.show()
fig.savefig('output/k7_verification_' + level + '_status.png', bbox_inches='tight')
plt.close('all')

### 41L dam level

In [ ]:
x = pd.to_datetime(df_real2.index.values).time
y1 = df_real2[level + ' Level'].values
y2 = df_sim[level + ' Level'].values

print('R squared = ', r2(y1, y2))
print('RMSE = ', rmse(y1, y2))

In [ ]:
fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)

x = df_sim['time_clean']

ax1.plot(x, df_real2[level + ' Level'], marker=None, label="Actual data", zorder=10)
ax1.plot(x, df_sim[level + ' Level'], marker=None, label="Simulation data", zorder=11, color=sns.color_palette('muted')[2])

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])

ax1.axvspan('2016-09-15 00:00:00','2016-09-15 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('2016-09-15 06:00:00','2016-09-15 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 07:00:00','2016-09-15 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 10:00:00','2016-09-15 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 18:00:00','2016-09-15 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 20:00:00','2016-09-15 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 22:00:00','2016-09-15 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x),max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)

fig.tight_layout()
plt.show()
fig.savefig('output/k7_verification_' + level + '_level.png', bbox_inches='tight')
plt.close('all')

### 31L pump status

In [ ]:
level = '31L'

x = pd.to_datetime(df_real2.index.values).time
y1 = df_real2[level + ' Status'].values
y2 = df_sim[level + ' Status'].values

print('R squared = ', r2(y1, y2))
print('RMSE = ', rmse(y1, y2))

In [ ]:
fig, ax1 = plt.subplots(figsize=(figure_size[0], figure_size[1]/1.5), dpi=1200)

x = df_sim['time_clean']

ax1.plot(x, df_real2[level + ' Status'], marker=None, label="Actual data", zorder=10)
ax1.plot(x, df_sim[level + ' Status'], marker=None, label="Simulation data", zorder=11, color=sns.color_palette('muted')[2])

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Number of pumps running')
ax1.set_ylim([0, 3])
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))

ax1.axvspan('2016-09-15 00:00:00','2016-09-15 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('2016-09-15 06:00:00','2016-09-15 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 07:00:00','2016-09-15 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 10:00:00','2016-09-15 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 18:00:00','2016-09-15 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 20:00:00','2016-09-15 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 22:00:00','2016-09-15 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x),max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)

fig.tight_layout()
plt.show()
fig.savefig('output/k7_verification_' + level + '_status.png', bbox_inches='tight')
plt.close('all')

### 31L dam level

In [ ]:
x = pd.to_datetime(df_real2.index.values).time
y1 = df_real2[level + ' Level'].values
y2 = df_sim[level + ' Level'].values

print('R squared = ', r2(y1, y2))
print('RMSE = ', rmse(y1, y2))

In [ ]:
fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)

x = df_sim['time_clean']

ax1.plot(x, df_real2[level + ' Level'], marker=None, label="Actual data", zorder=10)
ax1.plot(x, df_sim[level + ' Level'], marker=None, label="Simulation data", zorder=11, color=sns.color_palette('muted')[2])

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])

ax1.axvspan('2016-09-15 00:00:00','2016-09-15 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('2016-09-15 06:00:00','2016-09-15 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 07:00:00','2016-09-15 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 10:00:00','2016-09-15 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 18:00:00','2016-09-15 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 20:00:00','2016-09-15 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 22:00:00','2016-09-15 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x),max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)

fig.tight_layout()
plt.show()
fig.savefig('output/k7_verification_' + level + '_level.png', bbox_inches='tight')
plt.close('all')

### 20L pump status

In [ ]:
level = '20L'

x = pd.to_datetime(df_real2.index.values).time
y1 = df_real2[level + ' Status'].values
y2 = df_sim[level + ' Status'].values

print('R squared = ', r2(y1, y2))
print('RMSE = ', rmse(y1, y2))

In [ ]:
fig, ax1 = plt.subplots(figsize=(figure_size[0], figure_size[1]/1.5), dpi=1200)

x = df_sim['time_clean']

ax1.plot(x, df_real2[level + ' Status'], marker=None, label="Actual data", zorder=10)
ax1.plot(x, df_sim[level + ' Status'], marker=None, label="Simulation data", zorder=11, color=sns.color_palette('muted')[2])

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Number of pumps running')
ax1.set_ylim([0, 3])
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))

ax1.axvspan('2016-09-15 00:00:00','2016-09-15 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('2016-09-15 06:00:00','2016-09-15 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 07:00:00','2016-09-15 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 10:00:00','2016-09-15 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 18:00:00','2016-09-15 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 20:00:00','2016-09-15 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 22:00:00','2016-09-15 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x),max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)

fig.tight_layout()
plt.show()
fig.savefig('output/k7_verification_' + level + '_status.png', bbox_inches='tight')
plt.close('all')

### 20L dam level

In [ ]:
x = pd.to_datetime(df_real2.index.values).time
y1 = df_real2[level + ' Level'].values
y2 = df_sim[level + ' Level'].values

print('R squared = ', r2(y1, y2))
print('RMSE = ', rmse(y1, y2))

In [ ]:
fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)

x = df_sim['time_clean']

ax1.plot(x, df_real2[level + ' Level'], marker=None, label="Actual data", zorder=10)
ax1.plot(x, df_sim[level + ' Level'], marker=None, label="Simulation data", zorder=11, color=sns.color_palette('muted')[2])

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])

ax1.axvspan('2016-09-15 00:00:00','2016-09-15 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('2016-09-15 06:00:00','2016-09-15 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 07:00:00','2016-09-15 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 10:00:00','2016-09-15 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 18:00:00','2016-09-15 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 20:00:00','2016-09-15 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 22:00:00','2016-09-15 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x),max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)

fig.tight_layout()
plt.show()
fig.savefig('output/k7_verification_' + level + '_level.png', bbox_inches='tight')
plt.close('all')

### IPC pump status

In [ ]:
level = 'IPC'

x = pd.to_datetime(df_real2.index.values).time
y1 = df_real2[level + ' Status'].values
y2 = df_sim[level + ' Status'].values

print('R squared = ', r2(y1, y2))
print('RMSE = ', rmse(y1, y2))

In [ ]:
fig, ax1 = plt.subplots(figsize=(figure_size[0], figure_size[1]/1.5), dpi=1200)

x = df_sim['time_clean']

ax1.plot(x, df_real2[level + ' Status'], marker=None, label="Actual data", zorder=10)
ax1.plot(x, df_sim[level + ' Status'], marker=None, label="Simulation data", zorder=11, color=sns.color_palette('muted')[2])

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Number of pumps running')
ax1.set_ylim([0, 3])
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))

ax1.axvspan('2016-09-15 00:00:00','2016-09-15 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('2016-09-15 06:00:00','2016-09-15 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 07:00:00','2016-09-15 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 10:00:00','2016-09-15 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 18:00:00','2016-09-15 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 20:00:00','2016-09-15 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 22:00:00','2016-09-15 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x),max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)

fig.tight_layout()
plt.show()
fig.savefig('output/k7_verification_' + level + '_status.png', bbox_inches='tight')
plt.close('all')

### IPC dam level

In [ ]:
x = pd.to_datetime(df_real2.index.values).time
y1 = df_real2[level + ' Level'].values
y2 = df_sim[level + ' Level'].values

print('R squared = ', r2(y1, y2))
print('RMSE = ', rmse(y1, y2))

In [ ]:
fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)

x = df_sim['time_clean']

ax1.plot(x, df_real2[level + ' Level'], marker=None, label="Actual data", zorder=10)
ax1.plot(x, df_sim[level + ' Level'], marker=None, label="Simulation data", zorder=11, color=sns.color_palette('muted')[2])

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])

ax1.axvspan('2016-09-15 00:00:00','2016-09-15 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('2016-09-15 06:00:00','2016-09-15 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 07:00:00','2016-09-15 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 10:00:00','2016-09-15 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 18:00:00','2016-09-15 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 20:00:00','2016-09-15 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 22:00:00','2016-09-15 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x),max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)

fig.tight_layout()
plt.show()
fig.savefig('output/k7_verification_' + level + '_level.png', bbox_inches='tight')
plt.close('all')

### Surface dam level

In [ ]:
level = 'Surface'

x = pd.to_datetime(df_real2.index.values).time
y1 = df_real2[level + ' Level'].values
y2 = df_sim[level + ' Level'].values

print('R squared = ', r2(y1, y2))
print('RMSE = ', rmse(y1, y2))

In [ ]:
fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)

x = df_sim['time_clean']

ax1.plot(x, df_real2[level + ' Level'], marker=None, label="Actual data", zorder=10)
ax1.plot(x, df_sim[level + ' Level'], marker=None, label="Simulation data", zorder=11, color=sns.color_palette('muted')[2])

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])

ax1.axvspan('2016-09-15 00:00:00','2016-09-15 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('2016-09-15 06:00:00','2016-09-15 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 07:00:00','2016-09-15 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 10:00:00','2016-09-15 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 18:00:00','2016-09-15 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('2016-09-15 20:00:00','2016-09-15 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('2016-09-15 22:00:00','2016-09-15 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x),max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)

fig.tight_layout()
plt.show()
fig.savefig('output/k7_verification_' + level + '_level.png', bbox_inches='tight')
plt.close('all')

# Processing of x-factored simulation data

## 1-factor

In [ ]:
df = pd.read_csv('..\..\simulations\K7\output\K7_simulation_data_export_1-factor.csv.gz')
df['time_clean'] = pd.to_datetime(df['seconds'], unit='s')
df.head(5)

### Plot dam level and status

In [ ]:
level_name = '41L'

x = df['time_clean']

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

lines1 = ax1.plot(x, df[level_name + ' Level'])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax2 = ax1.twinx()
lines2 = ax2.plot(x, df[level_name + ' Status'], '--')

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])
ax2.set_ylabel('Number of pumps running')
ax2.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_ticks(np.arange(0, 101, 10))
ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x), max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)
#ax2.grid(which='major', alpha=0.5)
#ax1.minorticks_on()

p = lines1 + lines2
plt.legend(p, [level_name + ' dam level', level_name + ' pump status'],
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
#plt.legend(p,[p_.get_label() for subp in p for p_ in subp])

#plt.legend(lines1, ('a','b'), loc='best')
#plt.legend(lines2, ('c','d'), loc='best')

fig.tight_layout()
plt.show()
fig.savefig('output/k7_' + level + '_1_level_and_status.png', bbox_inches='tight')
plt.close('all')

In [ ]:
level_name = '31L'

x = df['time_clean']

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

lines1 = ax1.plot(x, df[level_name + ' Level'])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax2 = ax1.twinx()
lines2 = ax2.plot(x, df[level_name + ' Status'], '--')

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])
ax2.set_ylabel('Number of pumps running')
ax2.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_ticks(np.arange(0, 101, 10))
ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x), max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)
#ax2.grid(which='major', alpha=0.5)
#ax1.minorticks_on()

p = lines1 + lines2
plt.legend(p, [level_name + ' dam level', level_name + ' pump status'],
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
#plt.legend(p,[p_.get_label() for subp in p for p_ in subp])

#plt.legend(lines1, ('a','b'), loc='best')
#plt.legend(lines2, ('c','d'), loc='best')

fig.tight_layout()
plt.show()
fig.savefig('output/k7_' + level + '_1_level_and_status.png', bbox_inches='tight')
plt.close('all')

In [ ]:
level_name = '20L'

x = df['time_clean']

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

lines1 = ax1.plot(x, df[level_name + ' Level'])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax2 = ax1.twinx()
lines2 = ax2.plot(x, df[level_name + ' Status'], '--')

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])
ax2.set_ylabel('Number of pumps running')
ax2.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_ticks(np.arange(0, 101, 10))
ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x), max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)
#ax2.grid(which='major', alpha=0.5)
#ax1.minorticks_on()

p = lines1 + lines2
plt.legend(p, [level_name + ' dam level', level_name + ' pump status'],
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
#plt.legend(p,[p_.get_label() for subp in p for p_ in subp])

#plt.legend(lines1, ('a','b'), loc='best')
#plt.legend(lines2, ('c','d'), loc='best')

fig.tight_layout()
plt.show()
fig.savefig('output/k7_' + level + '_1_level_and_status.png', bbox_inches='tight')
plt.close('all')

In [ ]:
level_name = 'IPC'

x = df['time_clean']

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

lines1 = ax1.plot(x, df[level_name + ' Level'])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax2 = ax1.twinx()
lines2 = ax2.plot(x, df[level_name + ' Status'], '--')

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])
ax2.set_ylabel('Number of pumps running')
ax2.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_ticks(np.arange(0, 101, 10))
ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x), max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)
#ax2.grid(which='major', alpha=0.5)
#ax1.minorticks_on()

p = lines1 + lines2
plt.legend(p, [level_name + ' dam level', level_name + ' pump status'],
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
#plt.legend(p,[p_.get_label() for subp in p for p_ in subp])

#plt.legend(lines1, ('a','b'), loc='best')
#plt.legend(lines2, ('c','d'), loc='best')

fig.tight_layout()
plt.show()
fig.savefig('output/k7_' + level + '_1_level_and_status.png', bbox_inches='tight')
plt.close('all')

In [ ]:
level_name = 'Surface'

x = df['time_clean']

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

lines1 = ax1.plot(x, df[level_name + ' Level'])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)


ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])
ax1.yaxis.set_ticks(np.arange(0, 101, 10))
ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x), max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)
#ax2.grid(which='major', alpha=0.5)
#ax1.minorticks_on()

p = lines1
plt.legend(p, [level_name + ' dam level'],
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
#plt.legend(p,[p_.get_label() for subp in p for p_ in subp])

#plt.legend(lines1, ('a','b'), loc='best')
#plt.legend(lines2, ('c','d'), loc='best')

fig.tight_layout()
plt.show()
fig.savefig('output/k7_' + level + '_1_level_and_status.png', bbox_inches='tight')
plt.close('all')

### Plot power

#### Raw data

In [ ]:
fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

ax1.plot(df['time_clean'], df['Pump system total power'], marker=None, label='Power (raw data, 1 second)', zorder=10)
ax1.set_xlabel('Time of day')
ax1.set_ylabel('Power consumption (kW)')
#ax1.set_ylim([0, 4000])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x),max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)

fig.tight_layout()
plt.show()
fig.savefig('output/k7_1_power_raw.png', bbox_inches='tight')
plt.close('all')

#### Resampled data

In [ ]:
resampled = pd.DataFrame()
resampled["total_30_minute"] = df.set_index('time_clean')['Pump system total power'].resample('30T').mean()
resampled.to_csv('output/K7_resampled_power_1_factor.csv')

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

x = resampled.index
y = resampled['total_30_minute']

ax1.plot(x,y, 'x', label="Power (data points resampled to 30 minutes)", zorder=4)
ax1.plot(x,y, '--', label="Power (resampled, line plot)", zorder=2)
ax1.plot(x,y, drawstyle='steps-post', label="Power (resampled, step plot)", zorder=3)
ax1.set_xlabel('Time of day')
ax1.set_ylabel('Power consumption (kW)')
#ax1.set_ylim([0, 4000])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x),max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=1, mode="expand", borderaxespad=0.)

fig.tight_layout()
plt.show()
fig.savefig('output/k7_1_power_resampled_all.png', bbox_inches='tight')
plt.close('all')

In [ ]:
fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

x = resampled.index
y = resampled['total_30_minute']

ax1.plot(x,y, drawstyle='steps-post', label="Power (resampled, step plot)", zorder=3)
ax1.set_xlabel('Time of day')
ax1.set_ylabel('Power consumption (kW)')
#ax1.set_ylim([0, 4000])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x),max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=1, mode="expand", borderaxespad=0.)

fig.tight_layout()
plt.show()
fig.savefig('output/k7_1_power_resampled.png', bbox_inches='tight')
plt.close('all')

## 2-factor

In [ ]:
df = pd.read_csv('..\..\simulations\K7\output\K7_simulation_data_export_2-factor.csv.gz')
df['time_clean'] = pd.to_datetime(df['seconds'], unit='s')
df.head(5)

### Plot dam level and status

In [ ]:
level_name = '41L'

x = df['time_clean']

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

lines1 = ax1.plot(x, df[level_name + ' Level'])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax2 = ax1.twinx()
lines2 = ax2.plot(x, df[level_name + ' Status'], '--')

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])
ax2.set_ylabel('Number of pumps running')
ax2.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_ticks(np.arange(0, 101, 10))
ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x), max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)
#ax2.grid(which='major', alpha=0.5)
#ax1.minorticks_on()

p = lines1 + lines2
plt.legend(p, [level_name + ' dam level', level_name + ' pump status'],
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
#plt.legend(p,[p_.get_label() for subp in p for p_ in subp])

#plt.legend(lines1, ('a','b'), loc='best')
#plt.legend(lines2, ('c','d'), loc='best')

fig.tight_layout()
plt.show()
fig.savefig('output/k7_' + level + '_2_level_and_status.png', bbox_inches='tight')
plt.close('all')

In [ ]:
level_name = '31L'

x = df['time_clean']

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

lines1 = ax1.plot(x, df[level_name + ' Level'])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax2 = ax1.twinx()
lines2 = ax2.plot(x, df[level_name + ' Status'], '--')

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])
ax2.set_ylabel('Number of pumps running')
ax2.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_ticks(np.arange(0, 101, 10))
ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x), max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)
#ax2.grid(which='major', alpha=0.5)
#ax1.minorticks_on()

p = lines1 + lines2
plt.legend(p, [level_name + ' dam level', level_name + ' pump status'],
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
#plt.legend(p,[p_.get_label() for subp in p for p_ in subp])

#plt.legend(lines1, ('a','b'), loc='best')
#plt.legend(lines2, ('c','d'), loc='best')

fig.tight_layout()
plt.show()
fig.savefig('output/k7_' + level + '_2_level_and_status.png', bbox_inches='tight')
plt.close('all')

In [ ]:
level_name = '20L'

x = df['time_clean']

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

lines1 = ax1.plot(x, df[level_name + ' Level'])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax2 = ax1.twinx()
lines2 = ax2.plot(x, df[level_name + ' Status'], '--')

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])
ax2.set_ylabel('Number of pumps running')
ax2.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_ticks(np.arange(0, 101, 10))
ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x), max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)
#ax2.grid(which='major', alpha=0.5)
#ax1.minorticks_on()

p = lines1 + lines2
plt.legend(p, [level_name + ' dam level', level_name + ' pump status'],
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
#plt.legend(p,[p_.get_label() for subp in p for p_ in subp])

#plt.legend(lines1, ('a','b'), loc='best')
#plt.legend(lines2, ('c','d'), loc='best')

fig.tight_layout()
plt.show()
fig.savefig('output/k7_' + level + '_2_level_and_status.png', bbox_inches='tight')
plt.close('all')

In [ ]:
level_name = 'IPC'

x = df['time_clean']

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

lines1 = ax1.plot(x, df[level_name + ' Level'])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax2 = ax1.twinx()
lines2 = ax2.plot(x, df[level_name + ' Status'], '--')

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])
ax2.set_ylabel('Number of pumps running')
ax2.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_ticks(np.arange(0, 101, 10))
ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x), max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)
#ax2.grid(which='major', alpha=0.5)
#ax1.minorticks_on()

p = lines1 + lines2
plt.legend(p, [level_name + ' dam level', level_name + ' pump status'],
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
#plt.legend(p,[p_.get_label() for subp in p for p_ in subp])

#plt.legend(lines1, ('a','b'), loc='best')
#plt.legend(lines2, ('c','d'), loc='best')

fig.tight_layout()
plt.show()
fig.savefig('output/k7_' + level + '_2_level_and_status.png', bbox_inches='tight')
plt.close('all')

In [ ]:
level_name = 'Surface'

x = df['time_clean']

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

lines1 = ax1.plot(x, df[level_name + ' Level'])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)


ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])
ax1.yaxis.set_ticks(np.arange(0, 101, 10))
ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x), max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)
#ax2.grid(which='major', alpha=0.5)
#ax1.minorticks_on()

p = lines1
plt.legend(p, [level_name + ' dam level'],
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
#plt.legend(p,[p_.get_label() for subp in p for p_ in subp])

#plt.legend(lines1, ('a','b'), loc='best')
#plt.legend(lines2, ('c','d'), loc='best')

fig.tight_layout()
plt.show()
fig.savefig('output/k7_' + level + '_2_level_and_status.png', bbox_inches='tight')
plt.close('all')

### Plot power

In [ ]:
resampled = pd.DataFrame()
resampled["total_30_minute"] = df.set_index('time_clean')['Pump system total power'].resample('30T').mean()
resampled.to_csv('output/K7_resampled_power_2_factor.csv')

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

x = resampled.index
y = resampled['total_30_minute']

ax1.plot(x,y, drawstyle='steps-post', label='Power (resampled, step plot)', zorder=3)
ax1.set_xlabel('Time of day')
ax1.set_ylabel('Power consumption (kW)')
#ax1.set_ylim([0, 4000])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x),max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)

fig.tight_layout()
plt.show()
fig.savefig('output/k7_2_power_resampled.png', bbox_inches='tight')
plt.close('all')

## n-factor

In [ ]:
df = pd.read_csv('..\..\simulations\K7\output\K7_simulation_data_export_n-factor.csv.gz')
df['time_clean'] = pd.to_datetime(df['seconds'], unit='s')
df.head(5)

### Plot dam level and status

In [ ]:
level_name = '41L'

x = df['time_clean']

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

lines1 = ax1.plot(x, df[level_name + ' Level'])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax2 = ax1.twinx()
lines2 = ax2.plot(x, df[level_name + ' Status'], '--')

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])
ax2.set_ylabel('Number of pumps running')
ax2.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_ticks(np.arange(0, 101, 10))
ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x), max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)
#ax2.grid(which='major', alpha=0.5)
#ax1.minorticks_on()

p = lines1 + lines2
plt.legend(p, [level_name + ' dam level', level_name + ' pump status'],
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
#plt.legend(p,[p_.get_label() for subp in p for p_ in subp])

#plt.legend(lines1, ('a','b'), loc='best')
#plt.legend(lines2, ('c','d'), loc='best')

fig.tight_layout()
plt.show()
fig.savefig('output/k7_' + level + '_n_level_and_status.png', bbox_inches='tight')
plt.close('all')

In [ ]:
level_name = '31L'

x = df['time_clean']

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

lines1 = ax1.plot(x, df[level_name + ' Level'])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax2 = ax1.twinx()
lines2 = ax2.plot(x, df[level_name + ' Status'], '--')

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])
ax2.set_ylabel('Number of pumps running')
ax2.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_ticks(np.arange(0, 101, 10))
ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x), max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)
#ax2.grid(which='major', alpha=0.5)
#ax1.minorticks_on()

p = lines1 + lines2
plt.legend(p, [level_name + ' dam level', level_name + ' pump status'],
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
#plt.legend(p,[p_.get_label() for subp in p for p_ in subp])

#plt.legend(lines1, ('a','b'), loc='best')
#plt.legend(lines2, ('c','d'), loc='best')

fig.tight_layout()
plt.show()
fig.savefig('output/k7_' + level + '_n_level_and_status.png', bbox_inches='tight')
plt.close('all')

In [ ]:
level_name = '20L'

x = df['time_clean']

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

lines1 = ax1.plot(x, df[level_name + ' Level'])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax2 = ax1.twinx()
lines2 = ax2.plot(x, df[level_name + ' Status'], '--')

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])
ax2.set_ylabel('Number of pumps running')
ax2.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_ticks(np.arange(0, 101, 10))
ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x), max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)
#ax2.grid(which='major', alpha=0.5)
#ax1.minorticks_on()

p = lines1 + lines2
plt.legend(p, [level_name + ' dam level', level_name + ' pump status'],
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
#plt.legend(p,[p_.get_label() for subp in p for p_ in subp])

#plt.legend(lines1, ('a','b'), loc='best')
#plt.legend(lines2, ('c','d'), loc='best')

fig.tight_layout()
plt.show()
fig.savefig('output/k7_' + level + '_n_level_and_status.png', bbox_inches='tight')
plt.close('all')

In [ ]:
level_name = 'IPC'

x = df['time_clean']

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

lines1 = ax1.plot(x, df[level_name + ' Level'])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax2 = ax1.twinx()
lines2 = ax2.plot(x, df[level_name + ' Status'], '--')

ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])
ax2.set_ylabel('Number of pumps running')
ax2.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_ticks(np.arange(0, 101, 10))
ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x), max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)
#ax2.grid(which='major', alpha=0.5)
#ax1.minorticks_on()

p = lines1 + lines2
plt.legend(p, [level_name + ' dam level', level_name + ' pump status'],
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
#plt.legend(p,[p_.get_label() for subp in p for p_ in subp])

#plt.legend(lines1, ('a','b'), loc='best')
#plt.legend(lines2, ('c','d'), loc='best')

fig.tight_layout()
plt.show()
fig.savefig('output/k7_' + level + '_n_level_and_status.png', bbox_inches='tight')
plt.close('all')

In [ ]:
level_name = 'Surface'

x = df['time_clean']

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

lines1 = ax1.plot(x, df[level_name + ' Level'])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)


ax1.set_xlabel('Time of day')
ax1.set_ylabel('Dam level (%)')
ax1.set_ylim([0, 100])
ax1.yaxis.set_ticks(np.arange(0, 101, 10))
ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x), max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)
#ax2.grid(which='major', alpha=0.5)
#ax1.minorticks_on()

p = lines1
plt.legend(p, [level_name + ' dam level'],
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
#plt.legend(p,[p_.get_label() for subp in p for p_ in subp])

#plt.legend(lines1, ('a','b'), loc='best')
#plt.legend(lines2, ('c','d'), loc='best')

fig.tight_layout()
plt.show()
fig.savefig('output/k7_' + level + '_n_level_and_status.png', bbox_inches='tight')
plt.close('all')

### Plot power

In [ ]:
resampled = pd.DataFrame()
resampled["total_30_minute"] = df.set_index('time_clean')['Pump system total power'].resample('30T').mean()
resampled.to_csv('output/K7_resampled_power_n_factor.csv')

fig, ax1 = plt.subplots(figsize=figure_size, dpi=1200)
#fig.patch.set_facecolor('grey')

x = resampled.index
y = resampled['total_30_minute']

ax1.plot(x,y, drawstyle='steps-post', label='Power (resampled, step plot)', zorder=3)
ax1.set_xlabel('Time of day')
ax1.set_ylabel('Power consumption (kW)')
#ax1.set_ylim([0, 4000])

ax1.axvspan('1970-01-01 00:00:00','1970-01-01 05:59:59', alpha=.25, color=colour_op, zorder=0)
ax1.axvspan('1970-01-01 06:00:00','1970-01-01 06:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 07:00:00','1970-01-01 09:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 10:00:00','1970-01-01 17:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 18:00:00','1970-01-01 19:59:59', alpha=.25, color=colour_p, zorder=0)
ax1.axvspan('1970-01-01 20:00:00','1970-01-01 21:59:59', alpha=.25, color=colour_s, zorder=0)
ax1.axvspan('1970-01-01 22:00:00','1970-01-01 23:59:59', alpha=.25, color=colour_op, zorder=0)

ax1.xaxis.set_major_locator(mdates.HourLocator())
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=90)
ax1.set_xlim((min(x),max(x)))

ax1.grid(which='major', alpha=0.5)
#ax1.grid(which='minor', alpha=0.25)

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)

fig.tight_layout()
plt.show()
fig.savefig('output/k7_n_power_resampled.png', bbox_inches='tight')
plt.close('all')